In [6]:
import pymongo
from bson.json_util import dumps
uri = "mongodb://localhost:27017"
client = pymongo.MongoClient(uri)
db = client.dionaeareference

In [16]:
print(db.connections.find_one())
print(len(list(db.connections.find())))

{'_id': ObjectId('5d1d540605014d6d8686f118'), 'connection': 1, 'connection_type': 'accept', 'connection_transport': 'tcp', 'connection_protocol': 'epmapper', 'connection_timestamp': 1468414540, 'connection_root': 1, 'connection_parent': None, 'local_host': '202.43.92.50', 'local_port': 135, 'remote_host': '23.251.55.182', 'remote_hostname': '', 'remote_port': 2713}
500


In [116]:
pipeline = [
    {'$lookup': {
            'from': 'logins', 
            'localField': 'connection', 
            'foreignField': 'connection', 
            'as': 'login'
            }
    },
#     {'$unwind': { 
#                'path': '$login', 
#                 'preserveNullAndEmptyArrays': True 
#            }
#     },
    {'$lookup': {
        'from': 'mysql_commands', 
        'localField': 'connection', 
        'foreignField': 'connection', 
        'as': 'mysql_command'
        }
    },
    {'$lookup': {
        'from': 'mysql_command_ops', 
        'localField': 'mysql_command.mysql_command_cmd', 
        'foreignField': 'mysql_command_cmd', 
        'as': 'mysql_command_op'
        }
    },
    {'$project': {
         'login._id': 0, 
         'login.connection': 0, 
         'mysql_command._id': 0, 
         'mysql_command.connection': 0, 
         'mysql_command_op._id': 0
        }
    }
#     {'$project': {
#          '_id': '$_id',
# 'connection': '$connection',
# 'connection_parent': '$connection_parent',
# 'connection_protocol': '$connection_protocol',
# 'connection_root': '$connection_root',
# 'connection_timestamp': '$connection_timestamp',
# 'connection_transport': '$connection_transport',
# 'connection_type': '$connection_type',
# 'local_host': '$local_host',
# 'local_port': '$local_port',
# 'login': {
#     '$cond': {
#         'if': {'$eq': ['$login', []]},
#         'then': None,
#         'else': {
#             'login':'$login.login',
#             'login_username': '$login.login_username',
#             'login_password': '$login.login_password',
            
#         },
#     '$unwind': { 
#                'path': '$login', 
#                 'preserveNullAndEmptyArrays': True 
#            }
#     }
# },
# 'mysql_command': '$mysql_command',
# 'mysql_command_op': '$mysql_command_op',
# 'remote_host': '$remote_host',
# 'remote_hostname': '$remote_hostname',
# 'remote_port':'$remote_port '
#         }
#     },
#      {'$unwind': { 
#                'path': '$login', 
#                 'preserveNullAndEmptyArrays': True 
#            }
#     },
#     }
]
cari = db.connections.aggregate(pipeline)
from pprint import pprint
result = list(cari)
print(len(result))
pprint(result[0])

500
{'_id': ObjectId('5d1d540605014d6d8686f118'),
 'connection': 1,
 'connection_parent': None,
 'connection_protocol': 'epmapper',
 'connection_root': 1,
 'connection_timestamp': 1468414540,
 'connection_transport': 'tcp',
 'connection_type': 'accept',
 'local_host': '202.43.92.50',
 'local_port': 135,
 'login': [],
 'mysql_command': [],
 'mysql_command_op': [],
 'remote_host': '23.251.55.182',
 'remote_hostname': '',
 'remote_port': 2713}


In [118]:
for i in result:
    if i.get('mysql_command'):
        if len(i.get('mysql_command')) > 1:
            pprint(i)
            break

{'_id': ObjectId('5d1d540605014d6d8686f1c5'),
 'connection': 174,
 'connection_parent': None,
 'connection_protocol': 'mysqld',
 'connection_root': 174,
 'connection_timestamp': 1468428855,
 'connection_transport': 'tcp',
 'connection_type': 'accept',
 'local_host': '202.43.92.50',
 'local_port': 3306,
 'login': [{'login': 33, 'login_password': '', 'login_username': 'root'}],
 'mysql_command': [{'mysql_command': 5, 'mysql_command_cmd': 3},
                   {'mysql_command': 7, 'mysql_command_cmd': 1}],
 'mysql_command_op': [{'mysql_command_cmd': 1,
                       'mysql_command_op': 2,
                       'mysql_command_op_name': 'COM_QUIT'},
                      {'mysql_command_cmd': 3,
                       'mysql_command_op': 4,
                       'mysql_command_op_name': 'COM_QUERY'}],
 'remote_host': '45.114.106.190',
 'remote_hostname': '',
 'remote_port': 3751}


In [80]:
db.joincol500

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'dionaeareference'), 'joincol')

In [82]:
bulk_results = db.joincol500.insert_many(result , ordered=False)
# print(f"{bulk_results.modified_count} documents updated")